# 통계 기반 기법

## 말뭉치 전처리

In [4]:
# 1. 소문자 통일, 마침표 분리
text = "Yesterday my wife left me."
text = text.lower()
text = text.replace('.', ' .')
text

'yesterday my wife left me .'

In [5]:
# 2. 단어리스트 생성
words = text.split()
words

['yesterday', 'my', 'wife', 'left', 'me', '.']

In [6]:
# 3. 단어에 ID 부여, 딕셔너리 생성
word2id = {}
id2word = {}

for w in words:
    if w not in word2id:
        new_id = len(word2id)
        word2id[w] = new_id
        id2word[new_id] = w

print(word2id)
print(id2word)

{'yesterday': 0, 'my': 1, 'wife': 2, 'left': 3, 'me': 4, '.': 5}
{0: 'yesterday', 1: 'my', 2: 'wife', 3: 'left', 4: 'me', 5: '.'}


In [9]:
# 4. 단어 목록을 id목록으로 변경
import numpy as np
corpus = np.array([word2id[w] for w in words])
print(corpus)


[0 1 2 3 4 5]


In [13]:
# 한꺼번에
def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')
    word2id, id2word = {}, {}
    for w in words:
        if w not in word2id:
            new_id = len(word2id)
            word2id[w] = new_id
            id2word[new_id] = w
    corpus = np.array([word2id[w] for w in words])
    return corpus, word2id, id2word

text = 'I hope Sunday is a sunny day.'
for r in preprocess(text):
    print(r)

[0 1 2 3 4 5 6 7]
{'i': 0, 'hope': 1, 'sunday': 2, 'is': 3, 'a': 4, 'sunny': 5, 'day': 6, '.': 7}
{0: 'i', 1: 'hope', 2: 'sunday', 3: 'is', 4: 'a', 5: 'sunny', 6: 'day', 7: '.'}


## 동시발생 행렬

In [14]:
# 해당 단어 주변에 어떤 단어가 몇 번 등장했는지?: "통계기반 기법"
import sys
sys.path.append('..')
import numpy as np
from common.util import preprocess

text = "I hope Sunday is a sunny day."
corpus, word2id, id2word = preprocess(text)
print(corpus)
print(id2word)

[0 1 2 3 4 5 6 7]
{0: 'i', 1: 'hope', 2: 'sunday', 3: 'is', 4: 'a', 5: 'sunny', 6: 'day', 7: '.'}


In [17]:
# 윈도우 크기: 전후 몇개의 단어의 빈도를 셀 것인가
# 동시발생 행렬: 각 단어에 대해, 윈도우 크기 범위에서 다른 단어가 몇 번 나타나는가

def create_co_matrix(corpus, vocab_size, window_size=1):
    corpus_size = len(corpus) # 입력된 corpus의 길이. vocab_size와 혼동하지 않기
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)
    
    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left = idx - i
            right = idx + 1
            
            if left >= 0:
                left_id = corpus[left]
                co_matrix[word_id, left_id] += 1
            if right < corpus_size:
                right_id = corpus[right]
                co_matrix[word_id, right_id] += 1

    return co_matrix

corpus, word2id, id2word = preprocess("You say goodbye and I say hello.")
create_co_matrix(corpus, len(word2id), window_size=1)

array([[0, 1, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 1, 0],
       [0, 1, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0],
       [0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 1, 0]])

## 벡터 간 유사도

In [18]:
def cos_similarity(x, y, eps=1e-8):
    nx = x / (np.sqrt(np.sum(x ** 2)) + eps)
    ny = y / (np.sqrt(np.sum(y ** 2)) + eps)
    return np.dot(nx, ny)
    # eps: 인수가 영벡터일 때 0으로 나누지 않게끔 처리

실제 사용 예시는 동일 폴더 `similarity.py` 확인

## 유사 단어의 랭킹 표시

In [19]:
# 검색어와 비슷한 단어 순으로 표시
def most_similar(query, word2id, id2word, word_matrix, top=5):
    
    # 검색어 찾기
    if query not in word2id:
        print(f"단어 {query}을(를) 찾을 수 없습니다.")
        return
    query_id = word2id[query]
    query_vec = word_matrix[query_id]
    
    # 코사인유사도 계산
    vocab_size = len(word2id)
    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)
        
    # 내림차순 출력
    count = 0
    for i in (-1 * similarity).argsort():
        # argsort는 sort된 list가 아닌, 그 indices를 return
        # (-를 붙여줌으로써 역순으로 정렬할 수 있다.)
        if id2word[i] == query:
            continue
        print(f"{id2word[i]}: {similarity[i]}")
        count += 1
        if count >= top:
            return

실제 구현은 `most_similar.py` 파일을 살펴보자.

# 통계기반 기법 계산하기

## 점별 상호정보량 (pointwise mutual information)

$PMI(x, y) = log_2\frac{P(x, y)}{P(x)P(y)} = log_2\frac{\frac{C(x, y)}{N}}{\frac{C(x)}{N}\frac{C(y)}{N}} = log_2\frac{C(x,y)N}{C(x)C(y)}$
* $C$는 동시발생 행렬에서의 값을 뜻한다.
* 개별 단어가 단독으로 출현하는 횟수가 적을 때 점수가 더 높다.

$PPMI(x, y) = max(0, PMI(x, y))$
* $C(x, y) = 0$ 일 때 값이 $-\infty$가 되는 것을 방지한다.

In [20]:
# 동시발생행렬 -> PPMI 행렬
def ppmi(C, verbose=False, eps=1e-8):
    M = np.zeros_like(C, dtype=np.float32)
    N = np.sum(C)
    S = np.sum(axis=0)
    total = C.shape[0] * C.shape[1]
    count = 0
    
    for i in range(C.shape[0]):
        for j in range(C.shape[0]):
            pmi = np.log2(C[i, j] * N / (S[i] * S[j]) + eps)
            M[i, j] = max(0, pmi)
            
            if verbose:
                count += 1
                if count % (total // 100 + 1) == 0:
                    print(f"{100 * count / total:.1f}% 완료")
    
    return M

구현은 `ppmi.py`를 보세요.

## 차원 축소

* sparse matrix -> dense matrix
* 선대때 배운, Singular Value Decomposition ($A = U\Sigma V^T$)
* 직교행렬 $U$는 벡터공간의 기저를 형성한다
* $\Sigma$의 대각성분은 특잇값이 나열되어 있다 (축의 중요도 순서) -> 낮은 원소를 깎아내기

`count_method_small.py`를 보세요.


## PTB 데이터셋

* 더 큰 텍스트 데이터를 써 보자
* `<unk>`는 희소한 단어, `<eos>`는 end of sentence

`show_ptb.py`, `count_method_big.py`를 보세요.